In [ ]:
https://medium.com/@polanitzer/prediction-of-remaining-useful-life-of-an-engine-based-on-sensors-building-a-random-forest-in-ffad82c8a1c6

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor as GBR, RandomForestRegressor as RFR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/nasa_turbofan/fd001/train_FD001.csv')
X_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/nasa_turbofan/fd001/test_FD001.csv')
Y_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/regression/nasa_turbofan/fd001/test_rul_FD001.csv')
train

,unit_nr,time_cycles,setting_1,setting_2,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_15,s_17,s_20,s_21,RUL,breakdown
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8.4195,392,39.06,23.4190,191,0
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8.4318,392,39.00,23.4236,190,0
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8.4178,390,38.95,23.3442,189,0
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8.3682,392,38.88,23.3739,188,0
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8.4294,393,38.90,23.4044,187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,643.49,1597.98,1428.63,551.43,2388.19,9065.52,48.07,519.49,2388.26,8.4956,397,38.49,22.9735,4,0
20627,100,197,-0.0016,-0.0005,643.54,1604.50,1433.58,550.86,2388.23,9065.11,48.04,519.68,2388.22,8.5139,395,38.30,23.1594,3,0
20628,100,198,0.0004,0.0000,643.42,1602.46,1428.18,550.94,2388.24,9065.90,48.09,520.01,2388.24,8.5646,398,38.44,22.9333,2,0
20629,100,199,-0.0011,0.0003,643.23,1605.26,1426.53,550.68,2388.25,9073.72,48.39,519.67,2388.23,8.5389,395,38.29,23.0640,1,0


In [3]:
to_remove = ['unit_nr', 'breakdown', 'setting_1', 'setting_2']
output_column = ['RUL']
categorical = []
numerical = [col for col in train.columns if col not in categorical + output_column + to_remove]

In [ ]:
train.loc[train['unit_nr']==1, 'RUL']

0      191
1      190
2      189
3      188
4      187
      ... 
187      4
188      3
189      2
190      1
191      0
Name: RUL, Length: 192, dtype: int64

In [4]:
fig = go.Figure()
for i in numerical:
    fig.add_trace(go.Scatter(x=train.loc[train['unit_nr']==1, 'RUL'], y=train.loc[train['unit_nr']==1, i], mode='lines', name=i))
fig.add_trace(go.Scatter(x=train.loc[train['unit_nr']==1, 'RUL'], y=train.loc[train['unit_nr']==1, 'RUL'], mode='lines', name='RUL'))
fig.show()

In [5]:
data_train = train[train['time_cycles'] < 190]
data_valid = train[(train['time_cycles'] >= 190)]

data_train = data_train.sort_values(by='time_cycles').reset_index( drop = True )
data_valid = data_valid.sort_values(by='time_cycles').reset_index( drop = True )

features = categorical + numerical
 
# Creating the X, T and E inputs
X_train, Y_train = data_train[features], data_train[output_column].values.ravel()
X_valid, Y_valid = data_valid[features], data_valid[output_column].values.ravel()
Y_test = Y_test.values.ravel()

cat_enc_d = {}
for cat in categorical:
    OE = OneHotEncoder(sparse=False, drop='if_binary')
    train_ohe = OE.fit_transform(X_train[[cat]])
    valid_ohe = OE.transform(X_valid[[cat]])
    test_ohe = OE.transform(X_test[[cat]])
    for i in range(train_ohe.shape[1]):
        c = OE.categories_[0][i]
        X_train[cat+'_'+str(c)] = train_ohe[:, i]
        X_valid[cat+'_'+str(c)] = valid_ohe[:, i]
        X_test[cat+'_'+str(c)] = test_ohe[:, i]
    cat_enc_d[cat] = OE

X_train.drop(columns=categorical, inplace=True), X_valid.drop(columns=categorical, inplace=True),
X_test.drop(columns=categorical, inplace=True)

X_train.shape, X_valid.shape, X_test.shape

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



((18007, 14), (2624, 14), (13096, 17))

In [ ]:
X_train

,time_cycles,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_15,s_17,s_20,s_21
0,1,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8.4195,392,39.06,23.4190
1,1,642.26,1588.45,1400.02,552.94,2388.09,9065.41,47.61,521.18,2388.07,8.4300,392,38.81,23.4180
2,1,641.77,1583.59,1395.26,554.39,2387.98,9066.49,47.19,522.34,2388.00,8.4102,393,38.98,23.4468
3,1,641.96,1587.10,1400.65,553.68,2388.00,9054.19,47.34,522.03,2388.03,8.4094,392,38.87,23.4081
4,1,642.25,1589.89,1408.15,553.31,2388.07,9065.09,47.29,521.73,2388.11,8.4583,394,38.71,23.3029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18002,189,642.75,1584.23,1413.48,552.37,2388.19,9049.89,47.56,521.10,2388.16,8.4270,394,38.80,23.2674
18003,189,643.18,1582.91,1411.05,553.77,2387.99,9082.07,47.32,521.53,2387.96,8.4415,394,38.79,23.3599
18004,189,643.20,1601.34,1428.00,551.27,2388.26,9084.58,48.24,520.35,2388.20,8.5108,394,38.51,23.1160
18005,189,643.22,1604.68,1423.37,551.48,2388.19,9119.31,48.02,519.77,2388.22,8.4782,397,38.30,23.0607


In [15]:
hp = [1.00000000e+02, 6.29889436e-05, 1.40000000e+01, 4.70409885e-03,
       1.61605237e-01]
hp = [np.round(hp[0]).astype(int), hp[1], np.round(hp[2]).astype(int), hp[3], hp[4]]
regr = GBR(n_estimators=hp[0], min_samples_split=hp[1], max_features=hp[2], min_samples_leaf=hp[3],
               learning_rate=hp[4], random_state=1)
#regr = GBR(min_samples_leaf=6, max_depth=20, random_state=10)
regr.fit(X_train, Y_train)

GradientBoostingRegressor(learning_rate=0.161605237, max_features=14,
                          min_samples_leaf=0.00470409885,
                          min_samples_split=6.29889436e-05, random_state=1)

In [11]:
hp = [5.9e+01, 1.0e-05, 1.0e-05, 1.3e+01]
hp = [np.round(hp[0]).astype(int), hp[1], hp[2], np.round(hp[3]).astype(int)]
regr = RFR(n_estimators=hp[0], n_jobs=-1, min_samples_split=hp[1], min_samples_leaf=hp[2], max_features=hp[3],
               bootstrap=True, random_state=1)
regr.fit(X_train, Y_train)

RandomForestRegressor(max_features=13, min_samples_leaf=1e-05,
                      min_samples_split=1e-05, n_estimators=59, n_jobs=-1,
                      random_state=1)

In [16]:
pred = regr.predict(X_train)
r2_score(Y_train, pred), mean_squared_error(Y_train, pred, squared=False),  mean_absolute_error(Y_train, pred)

(0.6952962341439253, 36.92119236881184, 26.57960196156067)

In [17]:
pred = regr.predict(X_valid)
r2_score(Y_valid, pred), mean_squared_error(Y_valid, pred, squared=False),  mean_absolute_error(Y_valid, pred)

(0.8084610112936943, 15.684364003787866, 10.89470187557543)

In [18]:
last_cycle = X_test.groupby('unit_nr').last()[features]
pred = regr.predict(last_cycle)
r2_score(Y_test, pred), mean_squared_error(Y_test, pred, squared=False), mean_absolute_error(Y_test, pred)

(0.6230744843951788, 25.51276571370284, 18.348213735391347)

In [ ]:
last_cycle

,time_cycles,s_2,s_3,s_4,s_7,s_8,s_9,s_11,s_12,s_13,s_15,s_17,s_20,s_21
unit_nr,,,,,,,,,,,,,,
1,31,642.58,1581.22,1398.91,554.42,2388.08,9056.40,47.23,521.79,2388.06,8.4024,393,38.81,23.3552
2,49,642.55,1586.59,1410.83,553.52,2388.10,9044.77,47.67,521.74,2388.09,8.4505,391,38.81,23.2618
3,126,642.88,1589.75,1418.89,552.59,2388.16,9049.26,47.88,520.83,2388.14,8.4119,395,38.93,23.2740
4,106,642.78,1594.53,1406.88,552.64,2388.13,9051.30,47.65,521.88,2388.11,8.4634,395,38.58,23.2581
5,98,642.27,1589.94,1419.36,553.29,2388.10,9053.99,47.46,521.00,2388.15,8.4362,394,38.75,23.4117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,642.30,1590.88,1397.94,553.99,2388.03,9062.41,47.14,522.30,2388.01,8.4110,391,38.96,23.4606
97,134,642.59,1582.96,1410.92,554.05,2388.06,9076.36,47.38,521.58,2388.06,8.4500,395,38.61,23.2953
98,121,642.68,1599.51,1415.47,553.44,2388.13,9062.34,47.66,521.53,2388.09,8.4235,394,38.76,23.3608


In [10]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=last_cycle.index, y=Y_test, mode='lines', name='Y_test'))
fig.add_trace(go.Scatter(x=last_cycle.index, y=pred, mode='lines', name='PRED'))
fig.add_trace(go.Scatter(x=last_cycle.index, y=np.abs(Y_test-pred), mode='lines', name='DIFF'))

fig.show()

In [ ]:
Y_train

array([191, 136, 268, ...,   3,   9,  11])

In [ ]:
pd.DataFrame(data={'Y': Y_test, 'PRED': pred})

,Y,PRED
0,112,172.178139
1,98,144.959788
2,69,73.745551
3,82,106.871447
4,91,108.481859
...,...,...
95,137,171.764090
96,82,75.573215
97,59,94.404273
98,117,154.955122
